<br><br>
<h1 style="font-family:'Verdana',sans-serif; color:#1D7874; font-size:30px;">How a transmission system operates?</h1>

<br>
<h2 style="font-family:'Verdana',sans-serif; color:#1D7874;">Table of Contents</h2>
<br>
<ol style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
    <li ><a href='#introduction'>Introduction</a></li>
    <li ><a href='#motivation'>Motivation</a></li>
    <li><a href='#load_env'>Load an environment with pypownet</a></li>
    <li><a href='#grid_info'>Grid information</a></li>
    <li><a href='#grid'>The 4 substation grid</a></li>
    <li><a href='#line_cut'>A line disconnection and its effect</a></li>
    <li><a href='#config'>Inside a substation</a></li>
    <li><a href='#remedial'>A remedial solution</a></li>
    <li><a href='#conclusion'>Conclusion</a></li>
</ol>

<a id='introduction'></a>

<h3 style="font-family:'Verdana',sans-serif; color:#1D7874;">1. Introduction</h3>
<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
    This notebook is an introduction to power system operations. The operation of electrical power grids have been studied for many years around the world and involves many activities. Power grids, especially transmission grid, aims to transporting electricity over long distances from power plants to consumers. To be efficient and minimize energy losses, power grids often operate under high voltages (above 100kV). One of the main goals for power grid operators is to ensure a reliable and secure continuity of delivery of electricity to consumers under some additional quality criterion that can vary from one country to another.  
<br><br>
    For simplicity, a four substations grid is presented here with producers and consumers meshed through a grid. However, transmission grids are more complex and have many long lines that connect different cities or states. The grid often covers an entire country linking hundreds of power plants and distribution networks. But this should illustrate clearly enough how congestion could arise on the power grid and which operation can help manage it. Congestion management is actually the most critical and time-consuming task for a power grid operator.
<br><br> 
    <img src="https://i.ibb.co/fHm0Rb5/Example-Grid.jpg", width=650, ALIGN="middle", border=20>
<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:center; font-size:11px;">
(courtesy by Marvin Lerousseau)
</p>
</p>

<a id='motivation'></a>

<h3 style="font-family:'Verdana',sans-serif; color:#1D7874;">2. Motivation</h3>

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
    A power line has some physical capacity, aka thermal limit. Hence there is maximum power flow that is allowed through a line. Beyond it, a power line can break or can induce high risk for the safety of neighboring goods and people. We then say that a power line is congested, as illustrated by the power flow highlighted in yellow above. In that case, operators usually have only few minutes to reduce the flow in the congested line. It is very usual for electrical operators to set up the thermal limits above 80% of the line capacity as a warning signal to avoid undesired damages in the equipment.
<br><br>
    The first thing operators need to know are the power flows through the lines of the grid to anticipate and monitor congestions. Power flows can actually be computed through physical simulators, under what is called a load flow computation, given productions, consumptions and the topology (aka the graph) of the grid.
<br><br>
    As a first intuition, power flows increase when electrical consumption increases. More specifically, the flows in lines are affected by the dynamics of much electrical consumption and the availability of power plants or productions. It often varies along the course of a day, as consumer behaviors and usages change depending on the hour of the day: there are more activities in the middle of the day, while lighting is used mostly overnight and people finally get to sleep, leaving mostly electrical heating systems as well as some few industries in operations.   
<br><br>  
    As demand on power system increases over the years, the transmission system is more often forced to operate to its limits.  The congestion could be caused by day to day demand, but can also be due to some other factors such as topology grid changes by operator's actions, lines in maintenance or transient faults that can lead to cascading failures or put the system in an instability condition.
</p>

<a id='load_env'></a>

<h3 style="font-family:'Verdana',sans-serif; color:#1D7874;">3. Loading the environment</h3>

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
    Before starting the explanation, all libraries and the pypownet environment has to be loaded. The environment contains all essential information about the power grid such as the substations ids where the loads, generators and transmission lines are connected, the power flows and so on.
</p>

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from matplotlib.font_manager import FontProperties
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from reconstruct_grid import *

# Connect Plotly in offline mode. 
init_notebook_mode(connected = True) 
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import os
import pypownet.pypownet.environment
import pypownet.pypownet.runner

# Initialize the env.
environment = pypownet.environment.RunEnv(parameters_folder=os.path.abspath('data'),
                                          game_level='4_substations',
                                          chronic_looping_mode='natural', start_id=0,
                                          game_over_mode='hard' )

/!\ Using default reward signal, as reward_signal.py file is not found


<a id='grid_info'></a>

<h3 style="font-family:'Verdana',sans-serif; color:#1D7874;">4. Grid Information</h3>

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
Despide the grid is very small and simple, all information is accessible directly using the pypownet environment. Let's get some information about the most important objects over a power grid: power lines, electrical nodes, loads (aka consumptions) and generators (aka productions). 
<br><br>
    The state of those objects are very important in the reinforcement learning challenge as they will be used in observations. It means that given you observe the switch status, power flows, etc the agent should be able to take the correct action to keep flows in normal condition as possible. The following lines of code retrieve the nodes or substations ids and the numbers of elements connected to each busbar. 
</p>

In [3]:
# Load ids
loads_id = environment.action_space.loads_subs_ids
print ('Num of loads in total: {} -> Loads ids:'.format(len(loads_id), loads_id))
print (loads_id)

# Generators irds
gen_id = environment.action_space.prods_subs_ids
print ('Num of gen in total: {} -> Generators id:'.format(len(gen_id), gen_id))
print (gen_id)

# Line id sender
line_sender_id = environment.action_space.lines_or_subs_id
print ('Total transmission lines: {}'.format(len(line_sender_id)))
print ()
print ('Line sender id:')
print (line_sender_id)

# Line receiver id
line_rcv_id = environment.action_space.lines_ex_subs_id
print ('Line receiver id:')
print (line_rcv_id)
print ()

# Num of elements per SE
print ('Numbers of elements of subestation:')
print ('++  ++  ++  ++  ++  ++  ++  ++  ++')
for i in environment.action_space.substations_ids:
    print ('SE: {:d} - # elements: {}'.format(int(i), 
                                              environment.action_space.get_number_elements_of_substation(i)))


Num of loads in total: 3 -> Loads ids:
[2 3 4]
Num of gen in total: 2 -> Generators id:
[1 2]
Total transmission lines: 5

Line sender id:
[1 1 1 2 3]
Line receiver id:
[2 3 4 4 4]

Numbers of elements of subestation:
++  ++  ++  ++  ++  ++  ++  ++  ++
SE: 1 - # elements: 4
SE: 2 - # elements: 4
SE: 3 - # elements: 3
SE: 4 - # elements: 4


<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
Another important factor to control in grid operation is electrical losses. They are present due to some natural electricity physical behavior. They cause an imbalance between productions and consumptions. Losses are more pronounced in long lines because they offer more resistant to the current flows. The general formula is indicated below:
<br><br>
$$production = consumption + losses$$
<br>
You can get a general idea about losses taking the difference between generators and loads. Running one step in the environment you could get the observation given an action. For now, an action that does not change the status of the switch is passed (action_do_nothing), thus all production and consumptions information are retrieve it.
<br><br>
</p>

In [4]:
action_space = environment.action_space
observation_space = environment.observation_space
game = environment.game

# Create do_nothing action.
action_do_nothing = action_space.get_do_nothing_action()

In [5]:
# Run one step in the environment
obs, *_ = environment.step(action_do_nothing)
print (type(obs))

# for key, value in obs.items():
#      print(key, value)

<class 'numpy.ndarray'>


<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
As you see the observation is a numpy array. Sometimes is better to transform it as observation calling the following method with pypownet.
</p>

In [6]:
obs = observation_space.array_to_observation(obs)
vars(obs)

{'active_loads': array([11., 21., 51.]),
 'are_loads_cut': array([0., 0., 0.]),
 'loads_nodes': array([0., 0., 0.]),
 'active_productions': array([74.22710962, 16.        ]),
 'are_productions_cut': array([0., 0.]),
 'productions_nodes': array([0., 0.]),
 'lines_or_nodes': array([0., 0., 0., 0., 0.]),
 'lines_ex_nodes': array([0., 0., 0., 0., 0.]),
 'ampere_flows': array([ 9.27676801, 15.58783358, 18.49298615, 10.9899144 ,  3.02175015]),
 'lines_status': array([1., 1., 1., 1., 1.]),
 'timesteps_before_lines_reconnectable': array([0., 0., 0., 0., 0.]),
 'timesteps_before_planned_maintenance': array([0., 0., 0., 0., 0.]),
 'timesteps_before_lines_reactionable': array([0., 0., 0., 0., 0.]),
 'timesteps_before_nodes_reactionable': array([0., 0., 0., 0.]),
 'planned_active_loads': array([12., 22., 52.]),
 'planned_active_productions': array([123.,  60.]),
 'date_year': array([2018.]),
 'date_month': array([1.]),
 'date_day': array([2.]),
 'date_hour': array([0.]),
 'date_minute': array([5.]

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
The observation contains all essential information about the grid. For now, we will compute the losses in transmission lines taking the difference between what gets out from origin nodes and what gets into the extremity ones.

In [7]:
# Losses in transmission lines.
losses_in_lines = np.abs(obs.active_flows_origin) - np.abs(obs.active_flows_extremity)
print ('Losses in transmission lines {} MW'.format(losses_in_lines))
print ('Total sum losses in lines {:.4} MW'.format(sum(losses_in_lines)))

Losses in transmission lines [0.67209059 2.23216488 3.10941626 1.14496386 0.06853777] MW
Total sum losses in lines 7.227 MW


<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
The same value could be computed taking into account the formula described above.
</p>

In [8]:
print ('Total production {:.4} MW'.format(sum(obs.active_productions)))
print ('Total consumption {:.4} MW'.format(sum(obs.active_loads)))
print ('Losses in grid (prod - consump) {:.4} MW'.format(sum(obs.active_productions) - sum(obs.active_loads)))

Total production 90.23 MW
Total consumption 83.0 MW
Losses in grid (prod - consump) 7.227 MW


<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
Once reviewed how to obtain the grid information, It is time to execute actions. An action vector should be initialized beforehand. The vector will be modified it later with the desire action applied to a particular switch status in substations or lines.
<br><br>
    A random substation could be chosen first to create the corresponding Action Class which we named as applied action. The Class contains different methods to change the grid topology. 
</p>

In [9]:
# substation id.
sub_id = 2

# Create new_config array as do_nothing for 
# the current substation and an action array 
# as type action class.
new_config = np.zeros(action_space.get_number_elements_of_substation(sub_id))
applied_action = action_space.set_substation_switches_in_action(action_do_nothing, sub_id, new_config)
print (type(applied_action))

<class 'pypownet.game.Action'>


<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
The next function allows us to perform a few iterations using the step method in the environment given an action as an input and returns the observation variables as the power flow results, switches states, etc.
</p>

In [10]:
import copy

def sim(action, 
        t_action=0):
    
    # Restart all the game from the scratch.
    env = []
    env = pypownet.environment.RunEnv(parameters_folder=os.path.abspath('data'),
                                          game_level='4_substations',
                                          chronic_looping_mode='natural', start_id=0,
                                          game_over_mode='hard') 

    observation_space=env.observation_space
    
    # Iterating process..
    for i in range(1):
    
        # Execute action at step 0.
        if i == t_action: 
            obs_arry, *_ = env.step(action)
            obs = observation_space.array_to_observation(obs_arry)
            obs_action = copy.deepcopy(obs)
    
    return env, obs_action

<a id='grid'></a>

<h3 style="font-family:'Verdana',sans-serif; color:#1D7874;">5. A simple grid</h3>

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
The grid with four substations without any action is presented as follows. All power plants, consumptions and lines are on service and the load flows are indicating in each substation.
</p>

<div class="alert alert-warning">
    Note: How to interprate the flows?... for transmission lines (or) stands for origin and the convention is to take the flow as positive while (ex) means extremity and should have taken as negative. Thus one might guess the power flow direction.
</div>

In [11]:
layout = {'1':(-10, 10), '2':(10, 10), '3':(-10, -10), '4':(10, -10)}
label_pos = {'1':'top left', '2':'top right', '3':'bottom left', '4':'bottom right'}

In [12]:
# Run the game.
n_iter = 1

# Run the environment.
env, obs = sim(action_do_nothing)

# Get the grid.
grid_do_nothing = reconstruct_grid(env, obs, action_do_nothing, 
                                   layout, label_pos, size=(45,45), replace_dic=True)
iplot(grid_do_nothing)

/!\ Using default reward signal, as reward_signal.py file is not found


<h4 style="font-family:'Verdana',sans-serif; color:#1D7874;">How to interpret the results?</h4>

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
The power flows from one node to another through a power line. A positive flow on line A-B goes from A to B, a negative flow on line A-B goes from B to A. For instance, the direction of the flow in the line 1-4 gets out the substation 1 and gets in substation 4. 
</p>

<a id='line_cut'></a>

<h3 style="font-family:'Verdana',sans-serif; color:#1D7874;">6. Line disconnection</h3>

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
    Transmission lines link different states or regions between productions and consumptions. They are subject to sudden disconnections due, for instance, to thunderstorms. Since they operate over wide area, it is usual to have one or more lines out-of-service along the day because of thunder hits a line. 
<br><br>
    A disconnected line changes the configuration on the grid and causes a power flow's redistribution in the grid. Sometimes these undesired events lead to some overloads in other lines. You have not guess the reason why yet? The answer is very intuitive. When a thunder hits a line, the same amount of energy is demanded by customers but now you have one path less in the grid to transmit the energy, and if one via is affected the others will be loaded with more power.
<br><br>
    Substations are equipped with relays to disconnect a line whether a malfunctioning is detected. Normally the criteria to disconnect a line under an overload condition is to measure the flow and the time a line remains with some flow. Above a given threshold the relay order to open a line.
<br><br>
    This behavior occurs many times in electrical systems and the expertise and vast knowledge of operators help to alleviate the stress in order to guarantee energy to customers. A similar action could be replicated using pypownet. We simulate a disconnection in our small grid to see the resultant state of the grid.
</p>

<div class="alert alert-warning">
    Note: The user can modify and play with other lines.
</div>

In [13]:
# Specify the line to be disconnected.
id_line = 0

In [14]:
# Initialize action class with a ramdom substation (1)
new_config = np.zeros(action_space.get_number_elements_of_substation(1))
applied_action = action_space.set_substation_switches_in_action(action_do_nothing, 1, new_config)

# Apply the action (change switch status of a line)
action_space.set_lines_status_switch_from_id(action=applied_action,
                                             line_id=id_line,
                                             new_switch_value=1)

print ('New action vector')
print (applied_action.as_array())

New action vector
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]


<h4 style="font-family:'Verdana',sans-serif; color:#1D7874;">How the action vector is compossed?</h4>

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
    The action vector is no more than a concatenation over all arrays allowed to change the status to each element (on/off) in the grid. By default all elements are online before running the game. In the following figure you have a clear representation of the action vector for this particular case.
<br><br>
    <img src="http://i63.tinypic.com/fk8vn5.png", width=850, ALIGN="middle">
<br><br>
    But what really is an action?. An action in pypownet simulates what electrical operators do in control rooms. They monitor the grid in real time and can execute commands in a HMI (Human-Machine Interface) that are sent them to substations to open or close elements. The following figure try to illustrate this principle. 
<br><br>
    <img src="http://i68.tinypic.com/5edc1s.png", width=700, ALIGN="middle">
</p>

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
Once the action vector has been created, let's run the game to see how the grid is affected.
</p>

In [15]:
# Convert action vector Class into array.
applied_action = applied_action.as_array()
# Run simulation
env, obs = sim(applied_action)

# Plot the grid.
grid_do_nothing = reconstruct_grid(env, obs, applied_action, 
                                   layout, label_pos, size=(45,45), replace_dic=True)
iplot(grid_do_nothing)

/!\ Using default reward signal, as reward_signal.py file is not found


<h4 style="font-family:'Verdana',sans-serif; color:#1D7874;">Analyzing results?</h4>

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
In the previous results, we simulated a line disconnection originated by a lightning in the path 1-2. You immediately may notice overloads events over the lines 1-4 and 1-3. The resulting power flow, after the redistribution, causes a  flow increments over their thermal limits. In such condition, electrical operators have a few minutes to execute remedial actions to alleviate the system.
</p>

<a id='config'></a>

<h3 style="font-family:'Verdana',sans-serif; color:#1D7874;">7. Inside a Substation</h3>

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
    A power grid is composed of power lines connected through substations. Within a substation, there can be several arrangements between branches (lines, generators and loads) that allow them to meet. There exist many predefined configurations and each of them are better than others in terms of reliability.
<br><br>
    In this notebook, we will explore one type of configuration name as "double busbar".It is considered one of the most flexible and reliable configurations. It consists of two bars which are link together and branches could be connected to whatever of those. Substations in pypownet have this pre-defined configuration, which means that you can create to two electrical nodes per substations simulating a node-splitting.
<br><br>
    <img src="http://i65.tinypic.com/9s6y2s.png", width=800, ALIGN="middle">
<br><br>
    You can see in the first diagram is what is inside in a substation with a double busbar configuration. Regularly, all busbars are connected to each other. An action could split the busbars and pass some elements to the desired bar, thus a flow reconfiguration is possible.
</p>

<a id='remedial'></a>

<h3 style="font-family:'Verdana',sans-serif; color:#1D7874;">8. A Remedial solution</h3>

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
    When an overload is detected, operators have to execute a remedial action to alleviate the stress in the system. Normally the experience gained by years by people who work in electrical control rooms is a crucial factor to determine the best one. 
<br><br>
    Transmission networks are a bit complex with many paths where the power can flow (mesh grids). This also gives many alternatives to solve the congestions. In many countries changing the production is a valid solution. For instance, you can decrease the flow in the generators which are connected to the overloaded line and increase others that are near the consumptions.
<br><br>
    However, the previous one is costly, especially in countries where producers are market actors. An always cheap one is to play with the main grid configuration, thus one does not have to change the productions. These are the kind of actions RTE wants to build a new smart controllers for through the challenge. 
<br><br>
    To illustrate the issue, let's imagine the flow in the line 1-4 and 1-3 have increased by some external factor (such a lightning in the line 1-2). How one would solve the congestion with topological reconfiguration? Actually, one solution is to split the substation 4 and connect the line a selective path 1-2-4. Thus, a fictitious double line is created to supply the energy for load in substation 4.
<br><br>
    This action can be modeled with pypownet as well. In the next lines of code, you can see all the steps needed to perform the desired action and to get the grid with the power flow results. A new configuration layout should be provided by the user. For example, in the example described below, we execute an action to split the node 4 and connected the extremity with the line that comes from the node 1, line 2 and the respective load.
</p>

In [16]:
# SE id we want to modify.
sub_id = 4

# Change internal configuration of SE.
new_switch_config = [1, 1, 1, 0]

In [17]:
new_config = np.zeros(action_space.get_number_elements_of_substation(sub_id))
applied_action = action_space.set_substation_switches_in_action(action_do_nothing, sub_id, new_config)

In [18]:
#  Set new switches to the new state.
applied_action.set_substation_switches(sub_id, new_switch_config)

# See changes.
sub_i_new_config, sub_i_elm_type = applied_action.get_substation_switches(sub_id)
print ('New configuration of SE: {}'.format(sub_id))
print ('++   ++   ++   ++   ++   +')
for switch_status, elm_sub_type in zip(sub_i_new_config, sub_i_elm_type):
        print ('({}, {})'.format(switch_status, elm_sub_type.value))

New configuration of SE: 4
++   ++   ++   ++   ++   +
(1.0, consumption)
(1.0, extremity of power line)
(1.0, extremity of power line)
(0.0, extremity of power line)


In [19]:
# Run the environment.
env_after_action, obs_after_action = sim(applied_action.as_array())


# Plot the grid.
layout = {'1':(-10, 10), '2':(10, 10), '3':(-10, -10), '4':(5, -10),'6664':(10, -10)}
label_pos = {'1':'top left', '2':'top right', '3':'bottom left', '4':'bottom center','6664':'bottom right'}

grid_after_action = reconstruct_grid(env_after_action, 
                                     obs_after_action, 
                                     applied_action.as_array(), 
                                     layout, label_pos, size=(45,45), replace_dic=True)
iplot(grid_after_action)

/!\ Using default reward signal, as reward_signal.py file is not found


<a id='conclusion'></a>

<h3 style="font-family:'Verdana',sans-serif; color:#1D7874;">9. Conclusion</h3>

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
    In this notebook, we reviewed very quickly the main operation principles in transmission networks with a small grid. The continuity of the electric power service is indispensable due to the fact clients in transmission grids are majorly distribution networks or large industries. A desired line disconnection without an effective action that could solve the problem might bring blackouts and economic losses. 
<br><br>
    As we mentioned earlier, transmission operation involves many tasks but all of them are met to have one single objective: supply reliable and secure electricity. Operators work every day to minimize the impact of external factors that put the electrical system at risk but eventually when the grid is large and meshed, the daily operation is more complicated.
<br><br>
    The Pypownet platform was designed to interact together with the reinforcement learning approach. It allows executing actions in the transmission grid to manage lines with congestion without any cost involved. We covered all actions such as line disconnection and node splitting in the respective section. A user could play and execute more complicated ones.  
<br><br>
    The end state during the challenge is to train an agent able to learn a policy that could overcome obstacles that will be intentionally placed in some lines trying to reflect a real operation in transmission grids.
</p>